In [1]:
import tweepy
import json
import pandas as pd
import random
from dotenv import load_dotenv
import os
import time
from datetime import datetime
load_dotenv()

True

# Sentiment analysis de tweets respecto al plebiscito en Septiembre 2022 Chile

Este script busca en tweeter, con la los privilegios de la cuenta develop gratuita, los tweets según las palabras clave, la localización y las fechas determinadas.

La información mas importante de cada tweet es guardado en un dataframe.

La API libre de twitter tiene varias limitaciones, entre ella la fecha:

    Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
    
No se puede traer tweets mas antiguos que una semana. Ademas del peso sobre la cantidad de tweets, que cuelga la API por 15 minutos

##### Definición de palabras clave (_keywords_) a buscar dentro del texto de los tweets:

In [39]:
keywords_list = ['Constituyente',
'Convención Constituyente',
'Convención Constitucional',
'Asamblea Constituyente',
'plebiscito',
'referéndum',
'constitución chilena',
'Acuerdo por la paz social y la Nueva Constitución',
'Vamos por Chile',
'Unión Demócrata Independiente',
'Constituyentes Apartidistas',
'Independientes No Neutrales',
'Lista del Pueblo',
'Apruebo Dignidad',
'Lista del Apruebo',
'Pueblo Constituyente']

keywords_list_II = ['Chile Digno',
'Chile Unido',
'Unidos por Chile'
'Chile Libre',
'Coordinadora Constituyente Plurinacional y Popular',
'Somos región',
'Colectivo Socialista',
'Independientes por una Nueva Constitución',
'Movimientos sociales constituyentes',
'Colectivo del Apruebo',
'Wallmapur',
'Elisa Loncón',
'Campaña de desprestigio',
'rechazo del prebiscito de 2020',
'Carmen Gloria Valladares Moyano',
'Elisa Loncon',
'Destitucion de Elisa Loncon']

In [40]:
keywords_list = ["(" + word + ")" for word in keywords_list]

'#DestitucionDeElisaLoncon',
'juicio político',
'Jaime Bassa',
'Marcela Cubillos',
'Jaime Mañalich',
'Autodesprestigio',
'María Elisa Quinteros Cáceres',
'Gaspar Dominguez Donoso']

In [41]:
keywords_list  = ' OR '.join(keywords_list)

In [42]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#### Funciones
El script tiene dos funciones principales:
- Activación de la API con las credenciales
- Búsqueda de los tweets con lo parámetros definidos

##### Función Activación

In [154]:
# Función de generación de la API
def oauth():
    #Pass in our twitter API authentication key
    consumer_key = os.getenv('CONSUMER_KEY')
    consumer_secret = os.getenv('CONSUMER_SECRET')
    access_token = os.getenv('ACCESS_TOKEN')
    access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
    auth = tweepy.OAuth1UserHandler(
        consumer_key, consumer_secret,
        access_token, access_token_secret
    )
#     Instantiate the tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

##### Función Búsqueda

In [181]:
# Función de búsqueda de tweets según los keywords, cantidad y localización
def search(geo, search_query, page_limit):
    page_no = 1
    # Creamos las columnas para el DataFrame
    data = []
    dict = {}
    page_no = 1

    df = pd.DataFrame()
    columns = ['Time','User', 'Tweet']
    try:
        # Lanzamos la búsqueda
        pages = tweepy.Cursor(api.search_tweets,
                              q=search_query, 
                              tweet_mode='extended', 
                              geocode = geo).pages(page_limit)
        print(pages)
    # Iterate through the pages and print the text of the tweets
        for page in pages:    
            for i in range(len(page)):
                data.append([page[i].created_at, page[i].user.screen_name, page[i].full_text])
            page_no += 1
        df = pd.DataFrame(data, columns=columns)
        df['Time'] = df['Time'].dt.tz_localize(None) # Excel no soporta Datatime con zonas
        return df
    
    except BaseException as e:
        print('Status Failed On,',str(e))
        return

#### Ejecución 

##### Instanciamos la API

In [165]:
api = oauth()
api

#####  Cargamos los parámetros de búsqueda


- `sgo`: latitud y longitud de Santiago de Chile
- `radio`: radio en km alrededor del punto
- `no_of_tweets`: Cantidad de tweets para cada keyword
- `geo`: arma el feature según lo cargado anteriormente, con la ciudad y el radio.
- `keywords_list`: Se incluye en esta sección sólo para testear. Se limita a dos palabras. Si se comenta esa línea, usa toda la lista presentada arriba)

In [166]:
sgo = '-33.4489,-70.6693'
radio = '100km'
page_limit =2
geo = str(sgo) + ',' + str(radio) 
# keywords_list = ['Chile'] # Comentar esta linea para usar toda la keywords_list


In [167]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#####  Lanzamos la búsqueda

In [168]:
# df = pd.DataFrame()
df = search(geo,
            keywords_list,
            page_limit)


In [163]:
pd.set_option('display.max_colwidth', None)
df

,Time,User,Tweet
0,2022-08-24 18:04:07+00:00,chilesust,¡CONSTITUCIÓN! Gran día. la primera #ConstituciónEcológica de la Historia de #Chile y la primera #paritaria. Les invitamos a ser parte de la historia. Con esta Carta Magna tendremos muchas posibilidades de seguir creciendo hacia un futuro más sustentable para nuestrxs hijxs. https://t.co/F6IfYrijjp
1,2022-08-24 18:04:04+00:00,soldadito1951,Pero en Chile siguen subiendo !! https://t.co/UKSKuSFkUf
2,2022-08-24 18:03:58+00:00,InfinitaFM,"#Encuesta | Ante el gran rendimiento de Darío Osorio de la Universidad de Chile, en #SalimosJugandoEnCancha queremos saber:\n\n¿Cuál es el mejor escenario para Osorio al corto plazo?\n\nEscucha a @DiegoSanchezJ, @HugoMarcone, @gonzalofullu y @PatricioErland en https://t.co/r7wN21hZUP"
3,2022-08-24 18:03:57+00:00,PDI_CHILE,"[Ahora] En la Región del Biobío, PDI detuvo a Héctor Llaitul en virtud a orden judicial. En desarrollo."
4,2022-08-24 18:03:55+00:00,ChileanLeech,#Chile #Comunismo #comunistas #ComunismoEsMiseria #Nazbol https://t.co/fAAXv0pJAj
5,2022-08-24 18:03:46+00:00,AnibalMirandaRu,🖥💾📡🔌💡TECNO EN #COOPERATIVA 🇨🇱 #tecnologia #chile #cooperativa En Alemania debuta el primer tren de pasajeros con hidrógeno: https://t.co/Hye6i5Nrtd via @Cooperativa
6,2022-08-24 18:03:13+00:00,sebaalisboaf,@mvacarezza cuando vuelvas a vivir a Chile
7,2022-08-24 18:03:06+00:00,mellapolanco,Detienen a Héctor Llaitul en #Cañete. #Llaitul #Chile #Cam #Araucania https://t.co/sHV75sP7KV
8,2022-08-24 18:02:28+00:00,Bullamaniatic0s,¿Llegará al Clásico Universitario contra la UC? Luis Casanova volvió a jugar en la U tras dura lesión https://t.co/uTNtKX1PD7 https://t.co/gtOyvzSiY1
9,2022-08-24 18:02:17+00:00,janecarvajal,@MarcelaAninat Yo lo veo en CNN Chile


#####  Guardamos el dF

In [175]:
filename = 'plebiscito_' + datetime.now().strftime('%m_%d_%Y_%H:%M:%S') + '.xlsx'
# df.to_csv(filename, index = False)

In [180]:
df.to_excel(filename, index = False)